In [3]:
import os
import numpy as np
import pandas as pd
import json
import cv2

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

from keras import layers, models, optimizers, regularizers
from keras.applications import EfficientNetB0
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# import matplotlib.pyplot as plt

In [4]:
# test if running the GPU version of tensorflow
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [5]:
# loads directory of 3 datasets
dir_train = pd.read_csv('dataset/EuroSAT/train.csv')
dir_valid = pd.read_csv('dataset/EuroSAT/validation.csv')
dir_test = pd.read_csv('dataset/EuroSAT/test.csv')

In [6]:
# ## verify all 3 directories are equally sliced across 10 categories

# temp1 = dir_test['Label'].value_counts() / dir_test.shape[0]
# temp2 = dir_train['Label'].value_counts() / dir_train.shape[0]
# temp3 = dir_valid['Label'].value_counts() / dir_valid.shape[0]
# df_temp = pd.concat([temp1, temp2, temp3], axis=1)
# df_temp

# ## evenly sliced by category

In [7]:
img_dir = pd.concat([dir_train, dir_valid], ignore_index=False)
img_dir = img_dir.iloc[:,1:-1].reset_index().drop(['index'], axis=1)
print(img_dir.shape)
# img_dir = img_dir.iloc[:100, :] # limit sample size when testing
img_dir

(24300, 3)


,Filename,Label,ClassName
0,AnnualCrop/AnnualCrop_142.jpg,0,AnnualCrop
1,HerbaceousVegetation/HerbaceousVegetation_2835...,2,HerbaceousVegetation
2,PermanentCrop/PermanentCrop_1073.jpg,6,PermanentCrop
3,Industrial/Industrial_453.jpg,4,Industrial
4,HerbaceousVegetation/HerbaceousVegetation_1810...,2,HerbaceousVegetation
...,...,...,...
24295,SeaLake/SeaLake_1943.jpg,9,SeaLake
24296,AnnualCrop/AnnualCrop_211.jpg,0,AnnualCrop
24297,Industrial/Industrial_1428.jpg,4,Industrial
24298,AnnualCrop/AnnualCrop_2571.jpg,0,AnnualCrop


In [8]:
# Load images and labels/classes
images = []
classes = []
# labels.typeof()

# takes time. 3 mins on Geran's machine
base_path = 'dataset/EuroSAT/'
for index, row in img_dir.iterrows():
    img_path = os.path.join(base_path, row['Filename'])
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img)
    classes.append(row['ClassName'])

# Normalize images
images = np.array(images) / 255.0
labels = pd.get_dummies(classes).values

In [9]:
# ## use this if want to test code with only parts of the images. 
# images
# classes_orig = classes.copy()
# images_orig = images.copy()
# images = images[:1000]
# classes = classes[:1000]

In [10]:
# # find the min and max value of the list of 3-d arrays called images and return them in a tuple.

# import numpy as np

# def min_max(images):
#     min_value = np.min(images)
#     max_value = np.max(images)
#     return (min_value, max_value)

# min_max(images)

In [11]:
images.shape

(24300, 224, 224, 3)

In [12]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Set up the data augmentation
train_data_generator = ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)
valid_data_generator = ImageDataGenerator()

In [13]:
MODEL_PATH = "https://tfhub.dev/sayakpaul/convnext_base_21k_1k_224_fe/1"

# Define the pre-trained model to extract features
def get_feature_extractor(model_path=MODEL_PATH, res=224):
    hub_layer = hub.KerasLayer(model_path, trainable=False)
    model = keras.Sequential(
        [
            keras.layers.InputLayer((res, res, 3)),
            hub_layer
        ]
    )
    return model

feature_extractor = get_feature_extractor()

In [1]:
## Feature extractions done in batches

def predict_in_batches(model, data, batch_size=32):
    num_batches = int(np.ceil(len(data) / batch_size))
    features = []
    
    for batch_idx in range(num_batches):
        batch_start = batch_idx * batch_size
        batch_end = (batch_idx + 1) * batch_size
        
        batch_data = data[batch_start:batch_end]
        batch_features = model.predict(batch_data)
        features.extend(batch_features)
    
    return np.array(features)

# Set the desired batch_size
batch_size = 64

In [14]:
# Extract features from the training images
X_train_features = predict_in_batches(feature_extractor, X_train, batch_size)

1/2 [==============>...............] - ETA: 0s

ResourceExhaustedError: Graph execution error:

Detected at node 'activation/Gelu/truediv' defined at (most recent call last):
Node: 'activation/Gelu/truediv'
failed to allocate memory
	 [[{{node activation/Gelu/truediv}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_predict_function_31831]

In [ ]:
# Extract features from the testing images
X_test_features = predict_in_batches(feature_extractor, X_test, batch_size)

1/1 [==============================] - 1s 1s/step


In [ ]:
# Define the fine-tuning model
def get_fine_tuning_model(input_shape, num_classes=10):
    model = keras.Sequential(
        [
            keras.layers.InputLayer(input_shape),
            # layers.Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
            # layers.BatchNormalization(),
            # layers.Dropout(0.5),
            layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
            keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )
    return model

# Instantiate the fine-tuning model
input_shape = X_train_features.shape[1:]
fine_tuning_model = get_fine_tuning_model(input_shape)
fine_tuning_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 256)               262400    
                                                                 
 batch_normalization_1 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                2570      
                                                                 
Total params: 265,994
Trainable params: 265,482
Non-trainable params: 512
_________________________________________________________________


In [ ]:
# Compile the model
fine_tuning_model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the fine-tuning model
batch_size = 32
epochs = 1
validation_steps = len(X_test_features) // batch_size

history = fine_tuning_model.fit(X_train_features, y_train, validation_data=(X_test_features, y_test), batch_size=batch_size, epochs=epochs)

25/25 [==============================] - 1s 15ms/step - loss: 1.6552 - accuracy: 0.6388 - val_loss: 0.9293 - val_accuracy: 0.8300


In [ ]:
# # hub_layer = hub.KerasLayer(model_path, trainable=False)

# def get_model(model_path=MODEL_PATH, res=224, num_classes=10):
#     hub_layer = hub.KerasLayer(model_path, trainable=False)
#     model = keras.Sequential(
#         [
#             keras.layers.InputLayer((res, res, 3)),
#             hub_layer,
#             # layers.Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
#             # layers.BatchNormalization(),
#             # layers.Dropout(0.5),
#             layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
#             layers.BatchNormalization(),
#             layers.Dropout(0.5),
#             keras.layers.Dense(num_classes, activation="softmax"),
#         ]
#     )
#     return model

# model_1 = get_model()
# model_1.summary()

In [ ]:
# ## Compile the model. 
#
# ## TAKES TIME!! Unless using sebset of data, less epochs, etc. 
#
# model_1.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# # Train the model
# batch_size = 32
# epochs = 5
# validation_steps = len(X_test) // batch_size

# history = model_1.fit(train_data_generator.flow(X_train, y_train, batch_size=batch_size), validation_data=valid_data_generator.flow(X_test, y_test, batch_size=batch_size), validation_steps=validation_steps, epochs=epochs)

Epoch 1/5
608/608 [==============================] - 881s 1s/step - loss: 0.8132 - accuracy: 0.8666 - val_loss: 0.6546 - val_accuracy: 0.9077
Epoch 2/5
608/608 [==============================] - 860s 1s/step - loss: 0.5642 - accuracy: 0.9164 - val_loss: 0.5445 - val_accuracy: 0.9178
Epoch 3/5
608/608 [==============================] - 856s 1s/step - loss: 0.4757 - accuracy: 0.9241 - val_loss: 0.4681 - val_accuracy: 0.9191
Epoch 4/5
608/608 [==============================] - 856s 1s/step - loss: 0.4222 - accuracy: 0.9257 - val_loss: 0.4230 - val_accuracy: 0.9243
Epoch 5/5
608/608 [==============================] - 855s 1s/step - loss: 0.3895 - accuracy: 0.9281 - val_loss: 0.4646 - val_accuracy: 0.9067


In [ ]:
# model_1.save("model_1_nFull_epoch5_batch32")

INFO:tensorflow:Assets written to: model_1_nFull_epoch5_batch32\assets


INFO:tensorflow:Assets written to: model_1_nFull_epoch5_batch32\assets


In [ ]:
## if want to load later on: 
from tensorflow.keras.models import load_model
model_1 = load_model("model_1_nFull_epoch5_batch32")

## Evaluate model performance

In [ ]:
# Evaluate the model
batch_size = 32  # Choose a smaller batch size according to your GPU memory capacity
y_pred_batches = []

In [ ]:
for i in range(0, len(X_test_features), batch_size):
    batch_pred = fine_tuning_model.predict(X_test_features[i:i + batch_size])
    y_pred_batches.append(batch_pred)

y_pred = np.concatenate(y_pred_batches, axis=0)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# y_pred = model_1.predict(X_test)
# y_pred_classes = np.argmax(y_pred, axis=1)
# y_true = np.argmax(y_test, axis=1)

1/1 [==============================] - 0s 38ms/step


In [ ]:
print(classification_report(y_true, y_pred_classes))
print(confusion_matrix(y_true, y_pred_classes))

              precision    recall  f1-score   support

           0       0.92      0.88      0.90        25
           1       0.89      0.97      0.93        32
           2       0.95      0.79      0.86        24
           3       0.60      0.95      0.74        22
           4       1.00      0.86      0.92        14
           5       0.64      1.00      0.78        18
           6       0.92      0.75      0.83        16
           7       0.94      0.89      0.92        19
           8       0.88      0.33      0.48        21
           9       1.00      0.78      0.88         9

    accuracy                           0.83       200
   macro avg       0.87      0.82      0.82       200
weighted avg       0.86      0.83      0.82       200

[[22  0  0  1  0  1  1  0  0  0]
 [ 0 31  0  0  0  1  0  0  0  0]
 [ 0  2 19  0  0  3  0  0  0  0]
 [ 1  0  0 21  0  0  0  0  0  0]
 [ 0  0  0  2 12  0  0  0  0  0]
 [ 0  0  0  0  0 18  0  0  0  0]
 [ 0  0  1  1  0  1 12  1  0  0]
 [ 0  0  0

##### If you want to save the model: 

In [ ]:
# model_1.save("model_1")

# ## if want to load later on: 
# from tensorflow.keras.models import load_model
# loaded_model = load_model("model_1")

INFO:tensorflow:Assets written to: model_1\assets


INFO:tensorflow:Assets written to: model_1\assets


## Draft / Archive (Plz disregard)

In [ ]:
# import matplotlib.pyplot as plt # somehow this just doesnt work

In [ ]:
# testing/debugging
# print(classes[0])
# images[0].shape

AnnualCrop


(224, 224, 3)

In [ ]:
# ## Use this when running for 1st time
# MODEL_PATH = "https://tfhub.dev/sayakpaul/convnext_base_21k_1k_224_fe/1"
# model = hub.load(MODEL_PATH)
# # saved_model_path = 'model/'
# # tf.saved_model.save(model, saved_model_path)

# # ## Use this for subsequent runs when model already loaded locally
# # saved_model_path = 'model/'
# # model = tf.saved_model.load(saved_model_path)